In [1]:
import os

os.environ["NEO4J_URI"] = 'bolt://localhost:7687'
os.environ["NEO4J_USERNAME"] = 'neo4j'
os.environ["NEO4J_PASSWORD"] = 'password'
os.environ["OPENAI_API_KEY"] = 'sk-'

In [2]:
import dspy
from dspy.retrieve.neo4j_rm import Neo4jRM

# Configure LLM
lm = dspy.OpenAI(model="gpt-3.5-turbo")

# Configure Retriever
rm = Neo4jRM(
    index_name="vector",
    text_node_property="text"
)

# Configure DSPy to use the following language model and retrieval model by default
dspy.settings.configure(lm = lm, 
                        rm = rm)

/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Small training set with question and answer pairs
trainset = [dspy.Example(question="What were the two main things the author worked on before college?", 
                         answer="Writing and programming").with_inputs('question'),
            dspy.Example(question="What kind of writing did the author do before college?", 
                         answer="Short stories").with_inputs('question'),
            dspy.Example(question="What was the first computer language the author learned?", 
                         answer="Fortran").with_inputs('question'),
            dspy.Example(question="What kind of computer did the author's father buy?", 
                         answer="TRS-80").with_inputs('question'),
            dspy.Example(question="What was the author's original plan for college?", 
                         answer="Study philosophy").with_inputs('question'),]

In [4]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [5]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [6]:
from dspy.teleprompt import BootstrapFewShot

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=dspy.evaluate.answer_exact_match)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

100%|██████████| 5/5 [00:01<00:00,  3.30it/s]

Bootstrapped 1 full traces after 5 examples in round 0.


In [7]:
pred = compiled_rag(question = "Who is Ada Lovelace?")

In [8]:
pred

Prediction(
    context=['Augusta Ada King, Countess of Lovelace (née Byron; 10 December 1815 – 27 November 1852) was an English mathematician and writer, chiefly known for her work on Charles Babbage\'s proposed mechanical general-purpose computer, the Analytical Engine. She was the first to recognise that the machine had applications beyond pure calculation.\nAda Byron was the only legitimate child of poet Lord Byron and reformer Anne Isabella Milbanke. All Lovelace\'s half-siblings, Lord Byron\'s other children, were born out of wedlock to other women. Byron separated from his wife a month after Ada was born and left England forever.  He died in Greece when Ada was eight. Her mother was anxious about her upbringing and promoted Ada\'s interest in mathematics and logic in an effort to prevent her from developing her father\'s perceived insanity. Despite this, Ada remained interested in him, naming her two sons Byron and Gordon. Upon her death, she was buried next to him at her reques

In [21]:
lm.inspect_history(n=1)





Answer questions with short factoid answers.

---

Question: What was the author's original plan for college?
Answer: Study philosophy

Question: What were the two main things the author worked on before college?
Answer: Writing and programming

Question: What kind of computer did the author's father buy?
Answer: TRS-80

Question: What kind of writing did the author do before college?
Answer: Short stories

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: often between 1 and 5 words

---

Context:
[1] « the seventh one contained what many consider to be the first computer program—that is, an algorithm designed to be carried out by a machine. Other historians reject this perspective and point out that Babbage's personal notes from the years 1836/1837 contain the first programs for the engine. She also developed a vision of the capability of computers t